In [ ]:
!pip install -U -q PyDrive

In [ ]:
!pip install -U -q pyarrow

In [ ]:
# importações necessarias
import pandas as pd
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import shutil
warnings.filterwarnings('ignore')
import plotly.express as px
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Avião para a análise
airplane = "06120025"

In [ ]:
# Download dos dados para esta máquina virtual
shutil.unpack_archive(f"/content/drive/Shareddrives/Azul/drive-download-20230910T235638Z-002.zip", "./dataset")
shutil.unpack_archive(f"/content/dataset/06120028.zip", "./dataset")

In [ ]:
def convert_columns(df_filtered):
    for coluna in df_filtered.columns:
        if df_filtered[coluna].dtype == 'float64':
            df_filtered[coluna] = df_filtered[coluna].astype('float32')

        if df_filtered[coluna].dtype == 'int64':
            df_filtered[coluna] = df_filtered[coluna].astype('int32')

    return df_filtered

In [ ]:
filtered_columns = ['recording_time', 'dateDay-1', 'dateMonth-1', 'dateYear-1', 'phaseOfFlight-1',
                 'message0418DAA-1','message0422DAA-1',
                 'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b', 'bleedFavTmFbk-1a',
                 'bleedFavTmFbk-1b', 'bleedHprsovCmdStatus-1a',
                 'bleedHprsovCmdStatus-1b',
                  'bleedHprsovOpPosStatus-1a',
                 'bleedHprsovOpPosStatus-1b',
                  'bleedMonPress-1a',
                 'bleedMonPress-1b',
                 'bleedOnStatus-1a', 'bleedOnStatus-1b',
                 'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
                 'bleedPrsovClPosStatus-1a',
                 'bleedPrsovFbk-1a']

In [ ]:
dir_path = '/content/dataset/06120028'
print(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))

2385


In [ ]:
df = pd.DataFrame()
falha_index = []
df_number = 0
for _, _, arquivos in os.walk('/content/dataset/06120028/'):
    #nota: para alterar o tamanho do dataset, basta alterar o valor de 250. Recomendo utilizar o valor de 250 para não estourar o tamanho da RAM do colab
    while(df_number != 2168):
            try:
              path_air_falha = "/content/dataset/06120028/" + arquivos[df_number]
              df_read = pd.read_parquet(path_air_falha ,engine='auto')
              #df_filtered = df_read[filtered_cols]
              #df_converted = convert_columns(df_filtered)
              if len(df_read['message0418DAA-1'].value_counts()) > 1 or len(df_read['message0422DAA-1'].value_counts()) > 1:
                print(f'Erro: {df_read["dateDay-1"][3]}/{df_read["dateMonth-1"][3]}/{df_read["dateYear-1"][3]}')
                falha_index.append({
                    "path":path_air_falha,
                    "mes": int(df_read["dateMonth-1"][3]),
                    "dia": int(df_read["dateDay-1"][3]),
                    "ano": int(df_read["dateYear-1"][3])
                })
                print(f"{len(df_read['message0418DAA-1'].value_counts())} : {len(df_read['message0422DAA-1'].value_counts())} /n {df_number}")
            except:
              print("Erro ao ler o parquet")

            df_number = df_number + 1
    print("finsh: ",falha_index )

In [ ]:
falha_index

[{'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20220617031604.parquet',
  'mes': 6,
  'dia': 17,
  'ano': 2022},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20221212023042.parquet',
  'mes': 12,
  'dia': 12,
  'ano': 2022},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20221213165541.parquet',
  'mes': 12,
  'dia': 13,
  'ano': 2022},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20220617031602.parquet',
  'mes': 6,
  'dia': 16,
  'ano': 2022},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20220926111141.parquet',
  'mes': 9,
  'dia': 26,
  'ano': 2022},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20220708041541.parquet',
  'mes': 7,
  'dia': 8,
  'ano': 2022},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20230109153542.parquet',
  'mes': 1,
  'dia': 9,
  'ano': 2023},
 {'path': '/content/dataset/06120028/TCRF_ARCHIVE_06120028_20220927035102.parquet',
  'mes': 9,
  'dia': 27,
  'ano': 2022},


In [ ]:
interest_airplanes = [

]
for arquivo in falha_index:
  if int(arquivo["dia"]) > 7:
    interest_airplanes.append({
        "dia": int(arquivo["dia"])-7,
        "mes": int(arquivo["mes"]),
        "ano": int(arquivo["ano"])
    })

In [ ]:
interest_airplanes

In [ ]:
df = pd.DataFrame()
feature_index = []
index = 0
for _, _, arquivos in os.walk('/content/dataset/06120028'):
    #nota: para alterar o tamanho do dataset, basta alterar o valor de 250. Recomendo utilizar o valor de 250 para não estourar o tamanho da RAM do colab
    for i in arquivos:
      if (len(interest_airplanes)-1) >= index:
        df_read = pd.read_parquet("/content/dataset/06120028/"+i ,engine='auto')
        if not len(df_read['message0418DAA-1'].value_counts()) > 1 and not len(df_read['message0422DAA-1'].value_counts()) > 1:
          checkDay = int(df_read["dateDay-1"][3]) >= (interest_airplanes[index]["dia"]-5) and int(df_read["dateDay-1"][3]) <= (interest_airplanes[index]["dia"]+3)
          checkMonth = df_read["dateMonth-1"][3] == interest_airplanes[index]["mes"]
          checkYear = df_read["dateYear-1"][3] == interest_airplanes[index]["ano"]
          if checkDay and checkMonth and checkYear:
            print(f"Dia aceito: {df_read['dateDay-1'][3]} - Dia esperado: {str(interest_airplanes[index]['dia'])}")
            index += 1
            feature_index.append(i)
      else:
        break

feature_index

Dia aceito: 7.0 - Dia esperado: 10
Dia aceito: 6.0 - Dia esperado: 5
Dia aceito: 2.0 - Dia esperado: 6
Dia aceito: 12.0 - Dia esperado: 9
Dia aceito: 15.0 - Dia esperado: 19
Dia aceito: 4.0 - Dia esperado: 1
Dia aceito: 3.0 - Dia esperado: 2
Dia aceito: 21.0 - Dia esperado: 20
Dia aceito: 12.0 - Dia esperado: 10
Dia aceito: 19.0 - Dia esperado: 19
Dia aceito: 2.0 - Dia esperado: 3
Dia aceito: 4.0 - Dia esperado: 5
Dia aceito: 17.0 - Dia esperado: 19
Dia aceito: 8.0 - Dia esperado: 5
Dia aceito: 1.0 - Dia esperado: 5
Dia aceito: 20.0 - Dia esperado: 20
Dia aceito: 15.0 - Dia esperado: 19
Dia aceito: 12.0 - Dia esperado: 10
Dia aceito: 14.0 - Dia esperado: 11
Dia aceito: 4.0 - Dia esperado: 2
Dia aceito: 3.0 - Dia esperado: 1
Dia aceito: 7.0 - Dia esperado: 9
Dia aceito: 16.0 - Dia esperado: 20
Dia aceito: 11.0 - Dia esperado: 10
Dia aceito: 16.0 - Dia esperado: 20
Dia aceito: 6.0 - Dia esperado: 8
Dia aceito: 7.0 - Dia esperado: 8


['TCRF_ARCHIVE_06120028_20220607235040.parquet',
 'TCRF_ARCHIVE_06120028_20221207005142.parquet',
 'TCRF_ARCHIVE_06120028_20221202160843.parquet',
 'TCRF_ARCHIVE_06120028_20220612233540.parquet',
 'TCRF_ARCHIVE_06120028_20220917102005.parquet',
 'TCRF_ARCHIVE_06120028_20220704215041.parquet',
 'TCRF_ARCHIVE_06120028_20230103211242.parquet',
 'TCRF_ARCHIVE_06120028_20220921103201.parquet',
 'TCRF_ARCHIVE_06120028_20220612104302.parquet',
 'TCRF_ARCHIVE_06120028_20220920032803.parquet',
 'TCRF_ARCHIVE_06120028_20220602232339.parquet',
 'TCRF_ARCHIVE_06120028_20221204153143.parquet',
 'TCRF_ARCHIVE_06120028_20220917144439.parquet',
 'TCRF_ARCHIVE_06120028_20221208172341.parquet',
 'TCRF_ARCHIVE_06120028_20221201162441.parquet',
 'TCRF_ARCHIVE_06120028_20220920153141.parquet',
 'TCRF_ARCHIVE_06120028_20220917102001.parquet',
 'TCRF_ARCHIVE_06120028_20220612122738.parquet',
 'TCRF_ARCHIVE_06120028_20220614195940.parquet',
 'TCRF_ARCHIVE_06120028_20230104131940.parquet',
 'TCRF_ARCHIVE_06120

In [ ]:
feature_index

In [ ]:
df = pd.DataFrame()
for i in feature_index:
    df_read = pd.read_parquet("/content/dataset/06120028/"+i ,engine='auto')
    df_filtered = df_read[filtered_columns]
    df_converted = convert_columns(df_filtered)
    df = pd.concat([df, df_converted], axis=0)

for i in falha_index:
    df_read = pd.read_parquet(i["path"] ,engine='auto')
    df_filtered = df_read[filtered_columns]
    df_converted = convert_columns(df_filtered)
    df = pd.concat([df, df_converted], axis=0)

In [ ]:
df

In [ ]:
df_training = df.fillna(method="backfill")

In [ ]:
df_training.fillna(0)

In [ ]:
df_training = df_training.drop(labels=['dateYear-1','dateMonth-1','dateDay-1'], axis=1)

In [ ]:
df_training['message_error'] = df_training['message0418DAA-1'] + df_training['message0422DAA-1']


def substituir_valor(valor):
    if valor >= 2:
        return 1
    else:
        return valor

df_training['message_error'].apply(substituir_valor)
print(df_training['message_error'])

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9516    NaN
9517    NaN
9518    NaN
9519    NaN
9520    NaN
Name: message_error, Length: 5717679, dtype: float32


In [ ]:
df_training = df_training.drop(['message0418DAA-1','message0422DAA-1'], axis =1)

In [ ]:
df_training=df_training.dropna()

In [ ]:
df_training

,recording_time,phaseOfFlight-1,bleedFavTmCmd-1a,bleedFavTmCmd-1b,bleedFavTmFbk-1a,bleedFavTmFbk-1b,bleedHprsovCmdStatus-1a,bleedHprsovCmdStatus-1b,bleedHprsovOpPosStatus-1a,bleedHprsovOpPosStatus-1b,bleedMonPress-1a,bleedMonPress-1b,bleedOnStatus-1a,bleedOnStatus-1b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrsovClPosStatus-1a,bleedPrsovFbk-1a,message_error
0,0,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,42.375,42.375,0.0,0.0,-0.03125,0.03125,1.0,0.25,0.0
1,50,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,42.375,42.375,0.0,0.0,-0.03125,0.03125,1.0,0.25,0.0
2,100,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,42.375,42.375,0.0,0.0,-0.03125,0.03125,1.0,0.25,0.0
3,150,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,42.375,42.375,0.0,0.0,-0.03125,0.03125,1.0,0.25,0.0
4,200,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,42.375,42.375,0.0,0.0,-0.03125,0.03125,1.0,0.25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,475000,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,1.875,1.875,0.0,0.0,0.06250,0.06250,1.0,0.25,7.0
9501,475050,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,1.875,1.875,0.0,0.0,0.06250,0.06250,1.0,0.25,7.0
9502,475100,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,1.875,1.875,0.0,0.0,0.06250,0.06250,1.0,0.25,7.0
9503,475150,0.0,0.0,0.0,0.25,0.625,0.0,0.0,0.0,0.0,1.875,1.875,0.0,0.0,0.06250,0.06250,1.0,0.25,7.0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Crie um objeto MinMaxScaler
scaler = MinMaxScaler()

# Ajuste e transforme os dados
df_normalized = scaler.fit_transform(df_training)

In [ ]:
df_ready = pd.DataFrame(df_normalized, columns=df_training.columns)


In [ ]:
df_ready

In [ ]:
!pip install pycaret


In [ ]:
from pycaret.classification import *


setup(df_ready, target='message_error')


In [ ]:
model = create_model('model_name')


In [ ]:
compare_models()


In [ ]:
tuned_model = tune_model(model)


In [ ]:
evaluate_model(tuned_model)


In [ ]:
df_to_predict = pd.read_parquet('/content/dataset/06120028/TCRF_ARCHIVE_06120028_20220524142941.parquet' ,engine='auto')

In [ ]:
predictions = predict_model(tuned_model, data=df_to_predict)
